In [6]:
import findspark
findspark.init()

In [7]:
import pyspark
from pyspark.sql import SparkSession

In [8]:
from pyspark.sql import functions as F

In [9]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

22/03/07 07:20:22 WARN Utils: Your hostname, caio-HP-Probook resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface enp5s0)
22/03/07 07:20:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/07 07:20:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-06 13:34:03--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolvendo nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.79.36
Conectando-se a nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.79.36|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 733822658 (700M) [text/csv]
Salvando em: “fhvhv_tripdata_2021-02.csv”

fhvhv_tripdata_2021 100%[===================>] 699,83M  4,63MB/s    em 1m 58s  

2022-03-06 13:36:02 (5,91 MB/s) - “fhvhv_tripdata_2021-02.csv” salvo [733822658/733822658]



In [5]:
!wc -l fhvhv_tripdata_2021-02.csv

11908469 fhvhv_tripdata_2021-01.csv


In [6]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [7]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [8]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-02-01 00:10:40', dropoff_datetime='2021-02-01 00:21:09', PULocationID='35', DOLocationID='39', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-02-01 00:27:23', dropoff_datetime='2021-02-01 00:44:01', PULocationID='39', DOLocationID='35', SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime='2021-02-01 00:28:38', dropoff_datetime='2021-02-01 00:38:27', PULocationID='39', DOLocationID='91', SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime='2021-02-01 00:43:37', dropoff_datetime='2021-02-01 01:23:20', PULocationID='91', DOLocationID='228', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02872', pickup_datetime='2021-02-01 00:08:42', dropoff_datetime='2021-02-01 00:17:57', PULocationID='126', DOLocationID='250', SR_Flag=None)]

In [9]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [10]:
#UTILIZANDO APENAS O INÍCIO DO ARQUIVO PARA INFERIR OS TIPOS DE DADOS DAS COLUNAS
!head -n 1001 fhvhv_tripdata_2021-02.csv > head_2.csv

In [31]:
import pandas as pd

In [12]:
df_pandas = pd.read_csv('head_2.csv')

In [13]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [14]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [10]:
from pyspark.sql import types

In [16]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [17]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [18]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 2, 1, 0, 10, 40), dropoff_datetime=datetime.datetime(2021, 2, 1, 0, 21, 9), PULocationID=35, DOLocationID=39, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 2, 1, 0, 27, 23), dropoff_datetime=datetime.datetime(2021, 2, 1, 0, 44, 1), PULocationID=39, DOLocationID=35, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 2, 1, 0, 28, 38), dropoff_datetime=datetime.datetime(2021, 2, 1, 0, 38, 27), PULocationID=39, DOLocationID=91, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 2, 1, 0, 43, 37), dropoff_datetime=datetime.datetime(2021, 2, 1, 1, 23, 20), PULocationID=91, DOLocationID=228, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02872', pickup_datetime=d

In [19]:
#DIVIDINDO O ARQUIVO GRANDE EM PEDACOS MENORES, PARA SER PROCESSADO DE MANEIRA DISTRIBUIDA PELO CLUSTER SPARK
df = df.repartition(24)

In [20]:
df.write.parquet('fhvhv/2021/02/')

22/03/06 13:36:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
22/03/06 13:36:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
22/03/06 13:36:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
22/03/06 13:36:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
22/03/06 13:36:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
22/03/06 13:36:56 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 58,46% for 13 writers
22/03/06 13:36:56 WARN MemoryManager: Total allocation exceeds 95,

In [11]:
df = spark.read.parquet('fhvhv/2021/02/')

In [12]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [43]:
df_q3 = df \
   .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
   .withColumn('dropoff_date', F.to_date(df.pickup_datetime)) \
   .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID')
    
df_q3 \
    .filter(df_q3.pickup_date == '2021-02-15') \
    .count()


367170

In [61]:
df_q4 = df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn("diff_seconds", df.dropoff_datetime.cast(types.LongType()) - 
        df.pickup_datetime.cast(types.LongType()))

df_q4.registerTempTable('df_q4')

spark.sql("""
SELECT
    pickup_date,
    ROUND((max_duration / 60), 2) AS max_duration_minutes
FROM
    (SELECT
        pickup_date,
        MAX(diff_seconds) AS max_duration
    FROM
        df_q4
    GROUP BY
        pickup_date) q
ORDER BY
    max_duration DESC
""").show()

+-----------+--------------------+
|pickup_date|max_duration_minutes|
+-----------+--------------------+
| 2021-02-11|              1259.0|
| 2021-02-17|              953.68|
| 2021-02-20|              733.98|
| 2021-02-03|              677.55|
| 2021-02-19|              626.28|
| 2021-02-25|               583.5|
| 2021-02-18|              576.87|
| 2021-02-10|              569.48|
| 2021-02-21|              537.05|
| 2021-02-09|              534.78|
| 2021-02-06|              524.12|
| 2021-02-02|              515.22|
| 2021-02-05|              508.52|
| 2021-02-12|              502.47|
| 2021-02-08|              501.77|
| 2021-02-14|              496.28|
| 2021-02-22|               471.3|
| 2021-02-27|              452.83|
| 2021-02-15|              431.23|
| 2021-02-04|              426.53|
+-----------+--------------------+
only showing top 20 rows



In [65]:
df_q5 = df 

df_q5.registerTempTable('df_q5')

spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1) AS qty
FROM
    df_q5
GROUP BY
    dispatching_base_num
ORDER BY
    qty DESC
""").show()

+--------------------+-------+
|dispatching_base_num|    qty|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [1]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-03-07 07:15:48--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolvendo s3.amazonaws.com (s3.amazonaws.com)... 52.216.200.149
Conectando-se a s3.amazonaws.com (s3.amazonaws.com)|52.216.200.149|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 12322 (12K) [application/octet-stream]
Salvando em: “taxi+_zone_lookup.csv.1”

taxi+_zone_lookup.c 100%[===================>]  12,03K  --.-KB/s    em 0,002s  

2022-03-07 07:15:49 (6,92 MB/s) - “taxi+_zone_lookup.csv.1” salvo [12322/12322]



In [3]:
!wc -l taxi+_zone_lookup.csv

266 taxi+_zone_lookup.csv


In [4]:
!head taxi+_zone_lookup.csv

In [13]:
schema_lookup = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [15]:
df_q6 = df \
   .select('PULocationID', 'DOLocationID')

In [19]:
df_q6.show()

+------------+------------+
|PULocationID|DOLocationID|
+------------+------------+
|          94|          69|
|         205|         132|
|         159|         168|
|          62|          49|
|         234|         232|
|         152|          80|
|          75|          74|
|         241|         153|
|         157|         198|
|         236|         239|
|         143|         239|
|         174|         168|
|          26|          26|
|         149|         210|
|          80|          71|
|         228|          97|
|         229|         236|
|          79|         234|
|         229|          33|
|          10|         216|
+------------+------------+
only showing top 20 rows



In [16]:
df_q6.registerTempTable('df_q6')

In [22]:
df_lookup = spark.read \
    .option("header", "true") \
    .schema(schema_lookup) \
    .csv('taxi+_zone_lookup.csv')

In [23]:
df_lookup.registerTempTable('df_lookup')

In [24]:
df_lookup.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [34]:
df_result = spark.sql("""
SELECT
    pu.LocationID,
    do.LocationID,
    COALESCE(NULLIF(NULLIF(pu.Zone, 'NA'), 'NV'), 'Unknown') || ' / ' || COALESCE(NULLIF(NULLIF(do.Zone, 'NA'), 'NV'), 'Unknown') AS trip_itinerary,
    COUNT(1) AS qty
FROM
    df_q6 q6
INNER JOIN
    df_lookup pu
ON
    q6.PULocationID = pu.LocationID
INNER JOIN
    df_lookup do
ON
    q6.DOLocationID = do.LocationID
GROUP BY
    pu.LocationID,
    do.LocationID,
    trip_itinerary
ORDER BY
    qty DESC
""").toPandas()

#df_result.show()

In [37]:
df_result.head(50)

,LocationID,LocationID,trip_itinerary,qty
0,76,76,East New York / East New York,45041
1,26,26,Borough Park / Borough Park,37329
2,39,39,Canarsie / Canarsie,28026
3,61,61,Crown Heights North / Crown Heights North,25976
4,14,14,Bay Ridge / Bay Ridge,17934
5,129,129,Jackson Heights / Jackson Heights,14688
6,7,7,Astoria / Astoria,14688
7,42,42,Central Harlem North / Central Harlem North,14481
8,37,37,Bushwick South / Bushwick South,14424
9,89,89,Flatbush/Ditmas Park / Flatbush/Ditmas Park,13976
